In [ ]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when

from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
from sedona.core.enums import GridType
from sedona.core.enums import IndexType
from sedona.core.spatialOperator import JoinQueryRaw

In [ ]:
spark = SparkSession.builder.\
        master("local[*]").\
        appName("SedonaSQL-Example").\
        config("spark.serializer", KryoSerializer.getName).\
        config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
        config('spark.jars.packages',
               'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.0-incubating,'
               'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
        getOrCreate()
SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")

In [ ]:
census = ShapefileReader.readToGeometryRDD(sc, "data/drive-download-20220720T150603Z-001/census")
census_df = Adapter.toDf(census, spark)
census_df.createOrReplaceTempView("census")
census_df.printSchema()

subway = ShapefileReader.readToGeometryRDD(sc, "data/drive-download-20220720T150603Z-001/subway")
subway_df = Adapter.toDf(subway, spark)
subway_df.createOrReplaceTempView("subway")
subway_df.printSchema()

In [ ]:
result = spark.sql("SELECT Sum(popn_total) FROM census")
result.show()

In [ ]:
result1 = spark.sql("WITH distinct_blocks AS ( SELECT DISTINCT (blkid) popn_total FROM census JOIN subway ON ST_DWithin(census.geom, subway.geom, 500)) SELECT Sum(popn_total) FROM distinct_blocks")

In [ ]:
result_test = spark.sql("SELECT sum(popn_total) FROM census JOIN subway ON ST_DWithin(census.geometry, subway.geometry, 500)")